In [1]:
cd ~/Downloads

/Users/hamiltoncarpenter/Downloads


In [61]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk.data

In [3]:
cd 20_newsgroups/

/Users/hamiltoncarpenter/Downloads/20_newsgroups


In [4]:
ls

alt.atheism/              rec.autos/                sci.space/
comp.graphics/            rec.motorcycles/          soc.religion.christian/
comp.os.ms-windows.misc/  rec.sport.baseball/       talk.politics.guns/
comp.sys.ibm.pc.hardware/ rec.sport.hockey/         talk.politics.mideast/
comp.sys.mac.hardware/    sci.crypt/                talk.politics.misc/
comp.windows.x/           sci.electronics/          talk.religion.misc/
misc.forsale/             sci.med/


In [5]:
cd ..

/Users/hamiltoncarpenter/Downloads


In [6]:
cd all-the-news/

/Users/hamiltoncarpenter/Downloads/all-the-news


In [7]:
ls

articles1.csv* articles2.csv* articles3.csv*


In [8]:
df1 = pd.read_csv('articles1.csv')
df2 = pd.read_csv('articles2.csv')
df3 = pd.read_csv('articles3.csv')

In [9]:
df1pub = df1['publication']
df2pub = df2['publication']
df3pub = df3['publication']

In [10]:
df1pub.value_counts()

Breitbart           23781
CNN                 11488
New York Times       7803
Business Insider     6757
Atlantic              171
Name: publication, dtype: int64

In [11]:
df2pub.value_counts()

New York Post          17493
Atlantic                7008
National Review         6203
Talking Points Memo     5214
Guardian                4873
Buzzfeed News           4854
Fox News                4354
Name: publication, dtype: int64

In [12]:
df3pub.value_counts()

NPR                11992
Washington Post    11114
Reuters            10710
Vox                 4947
Guardian            3808
Name: publication, dtype: int64

In [13]:
Vox = 4947
#Buzzfeed_News = 854
the_atlantic = 7171
CNN = 11488
BuzzFeedNEWS = 4854


#Fox_news = 4354
National_Review = 6203
Breitbart = 23781
New_York_Post = 17493


In [50]:
Vox + the_atlantic +CNN#+BuzzFeedNEWS #Buzzfeed_News + 

23606

In [15]:
National_Review +Breitbart +New_York_Post #Fox_news + 

47477

In [16]:
#df_join = df1 +df2+df3

In [18]:
#df_join.shape

In [19]:
Breitbart_Articles = (df1[df1["publication"]=="Breitbart"])
National_Review_Articles = (df2[df2["publication"]=="National Review"])
New_York_Post_Articles = (df2[df2["publication"]=="New York Post"])


In [51]:
Vox_Articles = (df3[df3["publication"]=="Vox"])
Atlantic_Articles1 = (df1[df1["publication"]=="Atlantic"])
Atlantic_Articles = Atlantic_Articles1.append((df2[df2["publication"]=="Atlantic"]))
CNN_Articles = (df1[df1["publication"]=="CNN"])
#Buzzfeed_Articles = (df2[df2["publication"]=="Buzzfeed News"])



In [52]:
Conservative_Pubs = Breitbart_Articles
Conservative_Pubs = Conservative_Pubs.append(National_Review_Articles)
Conservative_Pubs = Conservative_Pubs.append(New_York_Post_Articles)

Liberal_Pubs = Vox_Articles
Liberal_Pubs = Liberal_Pubs.append(Atlantic_Articles)
Liberal_Pubs = Liberal_Pubs.append(CNN_Articles)

x=1

In [53]:
Liberal_Pubs.shape

(23614, 10)

In [54]:
Conservative_Pubs.shape

(47477, 10)

In [59]:
join_eval = Conservative_Pubs['publication']

In [60]:
join_eval.value_counts()

Breitbart          23781
New York Post      17493
National Review     6203
Name: publication, dtype: int64

In [64]:
con_text = Conservative_Pubs['content']
lib_text = Liberal_Pubs['content']

In [62]:
Conservative_Pubs.columns

Index(['Unnamed: 0', 'id', 'title', 'publication', 'author', 'date', 'year',
       'month', 'url', 'content'],
      dtype='object')

In [66]:
len(lib_text)

23614

In [71]:
lib_words = {}
con_words = {}
for i in range(0,len(lib_text)):
    lib_text[i] = str.split(lib_text[i])
for i in range(0,len(con_text)):
    con_text[i] = str.split(con_text[i])

for doc in lib_text.head():
    for word in doc:
        if word in lib_words:
            lib_words[word] = lib_words[word] +1
        else:
            lib_words[word] = 1

            
for doc in con_text.head():
    for word in doc:
        if word in con_words:
            con_words[word] = con_words[word] +1
        else:
            con_words[word] = 1

/Users/hamiltoncarpenter/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


KeyboardInterrupt: 

In [1]:
ls

FirstModel.ipynb  Untitled.ipynb    data/             submissions/
README.md         assessment1.md    images/           topics.md


In [2]:
pwd

'/Users/hamiltoncarpenter/galvanize/dsi-final-assessment'

In [3]:
pwd

'/Users/hamiltoncarpenter/galvanize/dsi-final-assessment'

In [4]:
ls

FirstModel.ipynb  Untitled.ipynb    data/             submissions/
README.md         assessment1.md    images/           topics.md
